## 用决策树来分类贷款是否优良

[LendingClub](https://www.lendingclub.com/) 是一家贷款公司. 在本次作业中,我们需要手动实现决策树来预测一份贷款是否安全，并对比不同复杂度下决策树的表现

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import tree
from sklearn import metrics




## 读取数据

In [2]:
#data = pd.read_csv(os.path.join("data", "loan_sub.csv"), sep=',')
data = pd.read_csv(os.path.join("../input", "loan_sub.csv"), sep=',')

## 打印可用特征

In [3]:
data.columns

## 预处理预测目标

预测目标是一列'bad_loans'的数据。其中**1**表示的是不良贷款，**0**表示的是优质贷款。

将预测目标处理成更符合直觉的标签，创建一列 `safe_loans`. 并且: 

* **+1** 表示优质贷款, 
* **-1** 表示不良贷款. 

In [4]:
# safe_loans =  1 => safe
# safe_loans = -1 => risky
#TODO

## 打印优质贷款与不良贷款的比例

In [5]:
data['safe_loans'].value_counts(normalize=True)

#### 这是一个不平衡数据， 好的贷款远比坏的贷款要多. 

## 选取用于预测的特征

In [7]:
cols = ['grade', 'term','home_ownership', 'emp_length']
target = 'safe_loans'

data = #TODO
data.head()

## 创建更为平衡的数据集  

* 对占多数的标签进行下采样  
* 注意有很多方法处理不平衡数据，下采样只是其中之一

In [8]:
data['safe_loans'].value_counts()


In [9]:

# use the percentage of bad and good loans to undersample the safe loans.
bad_ones = # TODO
safe_ones = # TODO
percentage = #TODO

risky_loans = bad_ones
safe_loans = #TODO

# combine two kinds of loans
data_set = #TODO

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [10]:
data_set[target].value_counts(normalize=True)

## Preprocessing your features

In [11]:
def dummies(data, columns=['pclass','name_title','embarked', 'sex']):
    for col in columns:
        data[col] = data[col].apply(lambda x: str(x))
        new_cols = [col + '_' + i for i in data[col].unique()]
        data = pd.concat([data, pd.get_dummies(data[col], prefix=col)[new_cols]], axis=1)
        del data[col]
    return data

In [12]:
#grade, home_ownership, target
cols = ['grade', 'term','home_ownership', 'emp_length']
data_set = #TODO
data_set.head()

## 将数据分成训练集和测试集

重要的事情说三遍!!  

**把你的爪子从TEST DATA上拿开!!**   
**把你的爪子从TEST DATA上拿开!!**  
**把你的爪子从TEST DATA上拿开!!**  


In [13]:
train_data, test_data = #TODO
trainX, trainY = #TODO
testX, testY = test_data[test_data.columns[1:]], pd.DataFrame(test_data[target])

## 建自己的决策树!  

任务：  
1 实现根据error来选择最佳划分特征的函数best_split()  
2 实现根据entropy来选择最佳特征的函数best_split_entropy()  
3 实现树节点的类TreeNode  
4 实现模型的类MyDecisionTree  

#### 任务1， 实现根据error来选择最佳划分特征的函数best_split()  
约定树的左边对应target == 0， 树的右边对应target == 1

In [46]:
def count_errors(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    
    positive_ones = #TODO
    negative_ones = #TODO
    
    return # TODO


def best_split(data, features, target):
    # return the best feature
    best_feature = None
    best_error = 2.0 
    num_data_points = float(len(data))  

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = # TODO
        
        # 右分支对应当前特征为1的数据点
        right_split = #TODO
        
        # 计算左边分支里犯了多少错
        left_misses = #TODO            

        # 计算右边分支里犯了多少错
        right_misses = #TODO
            
        # 计算当前划分之后的分类犯错率
        error = #TODO

        # 更新应选特征和错误率，注意错误越低说明该特征越好
        if error < best_error:
            best_error = #TODO
            best_feature = #TODO
    return best_feature

#### 任务2， 实现根据entropy来选择最佳特征的函数best_split_entropy()  


In [47]:
def entropy(labels_in_node):
    # 二分类问题: 0 or 1
    n = len(labels_in_node)
    s1 = (labels_in_node==1).sum()
    if s1 == 0 or s1 == n:
        return 0
    
    p1 = float(s1) / n
    p0 = 1 - p1
    return -p0 * np.log2(p0) - p1 * np.log2(p1)


def best_split_entropy(data, features, target):
    
    best_feature = None
    best_info_gain = float('-inf') 
    num_data_points = float(len(data))
    # 计算划分之前数据集的整体熵值
    entropy_original = #TODO

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = #TODO
        
        # 右分支对应当前特征为1的数据点
        right_split = #TODO 
        
        # 计算左边分支的熵值
        left_entropy = #TODO           

        # 计算右边分支的熵值
        right_entropy = #TODO
            
        # 计算左边分支与右分支熵值的加权和（数据集划分后的熵值）
        entropy_split = #TODO
        
        # 计算划分前与划分后的熵值差得到信息增益
        info_gain = #TODO

        # 更新最佳特征和对应的信息增益的值
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature
    return best_feature
    

#### 任务3，实现树节点的类TreeNode，每个树节点应该包含如下信息:  

   3.1 is_leaf: True/False  表示当前节点是否为叶子节点  
   
   3.2 prediction: 当前节点做全民公投的预测结果
   
   3.3 left: 左子树  
   
   3.4 right: 右子树 
   
   3.5 split_feature: 当前节点用来划分数据时所采用的特征

In [48]:
class TreeNode:
    def __init__(self, is_leaf, prediction, split_feature):
    # TODO
        
        

#### 任务4，实现模型的类MyDecisionTree， 实现如下主要函数  
  
  
   4.1 fit(): 模型在训练集上的学习  
   
   4.2 predict(): 模型在数据集上的预测
   
   4.3 score(): 模型在测试集上的得分   
   
   
   
   
   为了实现4.1 - 4.3的方法， 需要实现如下辅助函数  
   4.4 create_tree(): 创建一棵树  
   
   4.5 create_leaf(): 创建叶子节点  
   
   4.6 predict_single_data(): 模型预测单个数据  
   
   4.7 count_leaves(): 统计模型的叶子数

In [67]:
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
class MyDecisionTree(BaseEstimator):
    
    def __init__(self, max_depth, min_error):
        self.max_depth = max_depth
        self.min_error = min_error
    
    def fit(self, X, Y, data_weights = None):
        
        data_set = pd.concat([X, Y], axis=1)
        features = #TODO
        target = Y.columns[0]
        self.root_node = # TODO
        
        
    def predict(self, X):
        prediction = X.apply(lambda row: self.predict_single_data(self.root_node, row), axis=1)
        return prediction
        
        
    def score(self, testX, testY):
        target = # TODO
        result = self.predict(testX)
        return accuracy_score(testY[target], result)
    
    
    def create_tree(self, data, features, target, current_depth = 0, max_depth = 10, min_error=0):
        """
        探索三种不同的终止划分数据集的条件  
  
        termination 1, 当错误率降到min_error以下, 终止划分并返回叶子节点  
        termination 2, 当特征都用完了, 终止划分并返回叶子节点  
        termination 3, 当树的深度等于最大max_depth时, 终止划分并返回叶子节点
        """
        
    
        # 拷贝以下可用特征
        remaining_features = #TODO

        target_values = # TODO

        # termination 1
        if count_errors(target_values) <= min_error:
            print("Termination 1 reached.")     
            return # TODO

        # termination 2
        if len(remaining_features) == 0:
            print("Termination 2 reached.")    
            return # TODO    

        # termination 3
        if current_depth >= max_depth: 
            print("Termination 3 reached.")
            return # TODO



        # 选出最佳当前划分特征
        #split_feature = # TODO   #根据正确率划分
        split_feature = # TODO  # 根据信息增益来划分

        # 选出最佳特征后，该特征为0的数据分到左边，该特征为1的数据分到右边
        left_split = # TODO
        right_split = # TODO

        # 剔除已经用过的特征
        remaining_features = # TODO
        print("Split on feature %s. (%s, %s)" % (split_feature, str(len(left_split)), str(len(right_split))))

        # 如果当前数据全部划分到了一边，直接创建叶子节点返回即可
        if len(left_split) == len(data):
            print("Perfect split!")
            return self.create_leaf(left_split[target])
        if len(right_split) == len(data):
            print("Perfect split!")
            return self.create_leaf(right_split[target])

        # 递归上面的步骤
        left_tree = # TODO     
        right_tree = # TODO

        #生成当前的树节点
        result_node = TreeNode(False, None, split_feature)
        result_node.left = left_tree
        result_node.right = right_tree
        return result_node    
    
    
    
    def create_leaf(self, target_values):
        # 用于创建叶子的函数

        # 初始化一个树节点
        leaf = # TODO

        # 统计当前数据集里标签为+1和-1的个数，较大的那个即为当前节点的预测结果
        num_positive_ones = #TODO
        num_negative_ones = # TODO

        if num_positive_ones > num_negative_ones:
            leaf.prediction = 1
        else:
            leaf.prediction = -1

        # 返回叶子        
        return leaf 
    
    
    
    def predict_single_data(self, tree, x, annotate = False):   
        # 如果已经是叶子节点直接返回叶子节点的预测结果
        if tree.is_leaf:
            if annotate: 
                print("leaf node, predicting %s" % tree.prediction)
            return # TODO 
        else:
            # 查询当前节点用来划分数据集的特征
            split_feature_value = # TODO

            if annotate: 
                print("Split on %s = %s" % (tree.split_feature, split_feature_value))
            if split_feature_value == 0:
                #如果数据在该特征上的值为0，交给左子树来预测
                return # TODO
            else:
                #如果数据在该特征上的值为0，交给右子树来预测
                return # TODO    
    
    def count_leaves(self):
        return self.count_leaves_helper(self.root_node)
    
    def count_leaves_helper(self, tree):
        # TODO
    


In [68]:
m = MyDecisionTree(max_depth = 10, min_error = 1e-15)

In [69]:
m.fit(trainX, trainY)

In [70]:
m.score(testX, testY)

### 决策树复杂度的探讨

In [71]:
m.count_leaves()

#### 探索不同树深度对决策树的影响  

1 max_depth = 3  
2 max_depth = 7  
3 max_depth = 15


In [72]:
model_1 = # TODO
model_2 = # TODO
model_3 = # TODO


In [73]:
model_1.fit(trainX, trainY)
model_2.fit(trainX, trainY)
model_3.fit(trainX, trainY)

In [74]:
print("model_1 training accuracy :", model_1.score(trainX, trainY))
print("model_2 training accuracy :", model_2.score(trainX, trainY))
print("model_3 training accuracy :", model_3.score(trainX, trainY))

In [75]:
print("model_1 testing accuracy :", model_1.score(testX, testY))
print("model_2 testing accuracy :", model_2.score(testX, testY))
print("model_3 testing accuracy :", model_3.score(testX, testY))

In [76]:
print("model_1 complexity is: ", model_1.count_leaves())
print("model_2 complexity is: ", model_2.count_leaves())
print("model_3 complexity is: ", model_3.count_leaves())